In [1]:
# Python
import pandas as pd
from prophet import Prophet
from datetime import datetime
import holidays
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics

/home/nivag/2023-Linux/.python-3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
average_eut = pd.read_csv('sample-folder/IOH - Average of EUT.csv')

In [3]:
list1 = average_eut.columns[0:]
result = pd.DataFrame()


In [4]:
for x in range(2, len(average_eut.columns), 1):

    sitelist = average_eut[['Sector', 'Category']]
    duration_week = pd.DataFrame(data={'date': [list1[x]] * len(average_eut.index)})
    content = pd.DataFrame(average_eut.iloc[:,x]).reset_index(drop=True)
    content.rename({content.columns[0]:'EUT'}, axis=1, inplace=True)

    concat = pd.concat([sitelist, duration_week, content], axis=1)
    result = pd.concat([result, concat], axis=0)

In [5]:
result.reset_index(inplace=True, drop=True)
result['Year'] = result['date'].str.split('-WEEK').str[0]
result['WeekNumber'] = result['date'].str.split('-WEEK').str[1].astype(int)

result['StartOfWeek'] = pd.to_datetime(result['Year'], format='%Y') + pd.to_timedelta((result['WeekNumber'] - 1) * 7 + 1, unit='D')
result.drop(columns=['Year', 'WeekNumber'], inplace=True)

result.rename(columns={'EUT': 'y', 'StartOfWeek':'ds'}, inplace=True)

In [6]:
test = result[result['Sector'] == '11CLG0002_1'][['ds', 'y']]

In [7]:
model = Prophet()
model.fit(test)


15:51:24 - cmdstanpy - INFO - Chain [1] start processing
15:51:24 - cmdstanpy - INFO - Chain [1] done processing


In [8]:
future = model.make_future_dataframe(periods=7)

forecast = model.predict(future)
forecast = forecast[forecast['ds'] == '2023-06-12'][['ds', 'yhat']]

forecast.insert(2, 'Sector', 'siteA')

In [9]:
forecast

,ds,yhat,Sector
22,2023-06-12,6.221629,siteA


In [ ]:
# concatenate = pd.DataFrame()

# for x in result['Sector'].unique().tolist():
    
#     test = result[result['Sector'] == x][['ds', 'y']]
    
#     model = Prophet()
#     model.fit(test)

#     future = model.make_future_dataframe(periods=7)
    
#     forecast = model.predict(future)
#     forecast = forecast[forecast['ds'] == '2023-06-12'][['ds', 'yhat']]

#     forecast.insert(2, 'Sector', x)
#     print(x)
#     concatenate = pd.concat([concatenate, forecast], axis=0)

In [ ]:
concatenate.to_excel('result/prophet-result.xlsx')